In [102]:
import pandas as pd

pd.set_option("display.max_rows", 200)

In [103]:
constructor_standings = pd.read_csv("archive/constructor_standings.csv")
constructors = pd.read_csv("archive/constructors.csv")
races = pd.read_csv("archive/races.csv")

races = races[["raceId", "year", "date"]]

cons = constructor_standings.merge(constructors, on="constructorId")

In [104]:
cons.head()

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins,constructorRef,name,nationality,url
0,1,18,1,14.0,1,1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,7,19,1,24.0,1,1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
2,18,20,1,28.0,3,3,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
3,29,21,1,34.0,3,3,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
4,40,22,1,42.0,3,3,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren


In [105]:
features = ["points", "name", "constructorStandingsId", "wins", "raceId"]

cons = cons[features]

In [106]:
cons = cons.merge(races, on="raceId")

In [107]:
cons

,points,name,constructorStandingsId,wins,raceId,year,date
0,14.0,McLaren,1,1,18,2008,2008-03-16
1,8.0,BMW Sauber,2,0,18,2008,2008-03-16
2,9.0,Williams,3,0,18,2008,2008-03-16
3,5.0,Renault,4,0,18,2008,2008-03-16
4,2.0,Toro Rosso,5,0,18,2008,2008-03-16
...,...,...,...,...,...,...,...
12711,2.0,Brabham-Climax,20739,0,679,1967,1967-01-02
12712,6.0,Cooper-Climax,20736,0,679,1967,1967-01-02
12713,0.0,Lotus-BRM,20741,0,679,1967,1967-01-02
12714,0.0,LDS-Climax,20740,0,679,1967,1967-01-02


In [108]:
cons.dtypes

points                    float64
name                       object
constructorStandingsId      int64
wins                        int64
raceId                      int64
year                        int64
date                       object
dtype: object

In [109]:
cons["date"] = pd.to_datetime(cons.date)

In [110]:
current_constructors = ['Alfa Romeo','AlphaTauri','Alpine F1 Team','Aston Martin','Ferrari','Haas F1 Team',
                        'McLaren','Mercedes','Red Bull','Williams','Toro Rosso','Force India','Sauber','Racing Point',
                        'Renault']

cons = cons[cons.name.isin(current_constructors)].drop(columns=["constructorStandingsId", "raceId"])

In [111]:
cons

,points,name,wins,year,date
0,14.0,McLaren,1,2008,2008-03-16
2,9.0,Williams,0,2008,2008-03-16
3,5.0,Renault,0,2008,2008-03-16
4,2.0,Toro Rosso,0,2008,2008-03-16
5,1.0,Ferrari,0,2008,2008-03-16
...,...,...,...,...,...
12677,0.0,Aston Martin,0,1959,1959-12-12
12683,32.0,Ferrari,2,1959,1959-09-13
12693,0.0,Aston Martin,0,1959,1959-09-13
12695,28.0,Ferrari,2,1959,1959-08-23


In [117]:
def name_discretiser(r):
    if r == "Toro Rosso":
        return "AlphaTauri"
    if r == "Renault":
        return "Alpine F1 Team"
    if r == "Force India":
        return "Aston Martin"
    if r == "Racing Point":
        return "Aston Martin"
    if r == "Sauber":
        return "Alfa Romeo"
    else:
        return r
    
cons["constructor"] = cons.name.apply(name_discretiser)

In [116]:
cons.head()

,points,name,wins,year,date,constructor
0,14.0,McLaren,1,2008,2008-03-16,McLaren
2,9.0,Williams,0,2008,2008-03-16,Williams
3,5.0,Renault,0,2008,2008-03-16,Alpine F1 Team
4,2.0,Toro Rosso,0,2008,2008-03-16,AlphaTauri
5,1.0,Ferrari,0,2008,2008-03-16,Ferrari


In [112]:
import plotly.express as px

points = cons[(cons.year > 2015) & (cons.year < 2022)].groupby(["name", "year"]).points.max()